# CREATE INPUT FILES FROM DIABLO

these files will be used as input for DIABLO in R

In [46]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

In [53]:
import warnings
warnings.filterwarnings('ignore')

In [47]:
data_source = '../data/'

## RNASEQ

In [48]:
metadata_file = os.path.join(data_source, 'GSE98923_metadata.xlsx')

In [49]:
metadata = pd.read_excel(metadata_file, index_col=0, sheet_name='NO_REPLICATES')
metadata

,cultivar,year,state
MEAN SAMPLES,,,
CS_time0_2012,Cabernet Sauvignon,2012,green
CS_time1_2012,Cabernet Sauvignon,2012,green
CS_time2_2012,Cabernet Sauvignon,2012,green
CS_time3_2012,Cabernet Sauvignon,2012,green
CS_time4_2012,Cabernet Sauvignon,2012,green
...,...,...,...
PN_time7_2014,Pinot Noir,2014,mature
PN_time8_2014,Pinot Noir,2014,mature
PN_time9_2014,Pinot Noir,2014,mature


In [50]:
data_all_genes = pd.read_csv(os.path.join(data_source, 'GREAT_LOG_TPM_GSE98923_NOREPS.csv'), index_col=0)
data_all_genes = data_all_genes.transpose()
data_all_genes.shape

(73, 35336)

In [51]:
y_state = metadata['state']

In [52]:
trains_index = []
tests_index = []
ys_train = []

In [54]:
skf = StratifiedKFold(n_splits=5)
for i, (train_index, test_index) in enumerate(skf.split(data_all_genes, y_state)):
    
    X_train = data_all_genes.iloc[train_index, :]
    X_test = data_all_genes.iloc[test_index, :]

    y_train = y_state.iloc[train_index]
    y_test = y_state.iloc[test_index]
    
    ys_train.append(y_train)
    
    trains_index.append(train_index)
    tests_index.append(test_index)
    
    # remove some features
    vt = VarianceThreshold(0.1)
    filter_train = vt.fit(X_train)
    
    train_filtered = filter_train.transform(X_train)
    test_filtered = filter_train.transform(X_test)
    
    cols_inds = vt.get_support(indices=True)
    
    X_train_filtered = pd.DataFrame(train_filtered, index=X_train.index, columns=X_train.columns[cols_inds])
    X_test_filtered = pd.DataFrame(test_filtered, index=X_test.index, columns=X_test.columns[cols_inds])
    
    kb2 = SelectKBest(f_classif, k=500)

    kb2_fit = kb2.fit(X_train_filtered, y_train)

    train_filtered2 = kb2_fit.transform(X_train_filtered)
    test_filtered2 = kb2_fit.transform(X_test_filtered)

    cols_inds = kb2_fit.get_support(indices=True)

    X_train_filtered2 = pd.DataFrame(train_filtered2, columns=X_train_filtered.columns[cols_inds], index=X_train_filtered.index)
    X_test_filtered2 = pd.DataFrame(test_filtered2, columns=X_test_filtered.columns[cols_inds], index=X_test_filtered.index)
    
    scaler_model = StandardScaler().fit(X_train_filtered2)
    X_train_scaled = scaler_model.transform(X_train_filtered2)
    X_test_scaled = scaler_model.transform(X_test_filtered2)
    
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered2.columns, index=X_train_filtered2.index)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered2.columns, index=X_test_filtered2.index)
    
    X_train_scaled_df.to_csv(os.path.join(data_source, 'XTRAIN_RNASEQ_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    X_test_scaled_df.to_csv(os.path.join(data_source, 'XTEST_RNASEQ_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    y_train.to_csv(os.path.join(data_source, 'yTRAIN_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))
    
    y_test.to_csv(os.path.join(data_source, 'yTEST_ALL_GENES_NOREPS_SPLIT_' + str(i) + '.csv'))

## METABOLOMICS

In [55]:
metadata_noreps = pd.read_excel(os.path.join(data_source, 'metabolomics_metadata.xlsx'), sheet_name='CONVERSION', index_col=0)

In [56]:
data_reps = pd.read_excel(os.path.join(data_source, 'metabolomics.xlsx'), index_col=0, header=0)
data_reps = data_reps.loc[:, data_reps.columns != 'Method']
data_reps = data_reps.transpose()
data_reps.head()

Metabolite Name,xylose,xylonic_acid,vanillic_acid,valine,urea,uracil,UDP-glucuronic_acid,tyrosol,tyrosine_gc,tryptophan,...,proline,quercetin-3-glucoside,quercetin-3-glucuronide,resveratrol,resveratrol_dimer_(pallidol_or_viniferin),spirotetramat,splitomicin,taxifolin,tributyl_phosphate,tyrosine
CS12_0_1_0,3.865952,0.308311,0.158177,1.568365,0.541555,0.075067,0.635389,0.160858,1.193029,4.490617,...,108.618224,1.062913,21.228368,0.000955,0.0,6.103717,0.0,0.0,34.438299,0.000000
CS12_0_2_0,3.260054,0.243968,0.144772,2.469169,1.083110,0.104558,0.493298,0.067024,1.495979,4.965147,...,95.041575,0.866498,24.132731,0.000000,0.0,10.381482,0.0,0.0,152.884714,0.000000
CS12_0_3_0,4.067024,0.308311,0.048257,1.152815,2.024129,0.080429,0.823056,0.104558,1.474531,4.353887,...,92.855773,0.840128,16.748935,0.000000,0.0,15.239863,0.0,0.0,38.786396,0.421625
CS12_A_1_1,3.549598,0.249330,0.115282,0.640751,1.659517,0.045576,0.418231,0.000000,0.600536,2.530831,...,81.820920,0.938593,18.445534,0.000000,0.0,0.412186,0.0,0.0,190.192118,0.000000
CS12_A_1_10,2.364611,1.664879,0.112601,8.978552,1.694370,0.142091,2.962466,0.099196,1.766756,0.597855,...,206.592111,16.024570,10.791648,0.000000,0.0,8.410129,0.0,0.0,55.184038,0.000000


In [57]:
data_mets_noreps = data_reps.groupby(metadata_noreps['groups']).mean()
data_mets_noreps.head()

Metabolite Name,xylose,xylonic_acid,vanillic_acid,valine,urea,uracil,UDP-glucuronic_acid,tyrosol,tyrosine_gc,tryptophan,...,proline,quercetin-3-glucoside,quercetin-3-glucuronide,resveratrol,resveratrol_dimer_(pallidol_or_viniferin),spirotetramat,splitomicin,taxifolin,tributyl_phosphate,tyrosine
groups,,,,,,,,,,,,,,,,,,,,,
CS_time0_2012,3.731010,0.286863,0.117069,1.730116,1.216265,0.086685,0.650581,0.110813,1.387846,4.603217,...,98.838524,0.923180,20.703345,0.000318,0.000000,10.575021,0.0,0.000000,75.369803,0.140542
CS_time0_2013,3.248436,0.571939,0.219839,0.758713,1.244861,0.202860,1.277033,0.256479,1.069705,1.967828,...,1040.845728,12.307100,77.785800,1.187485,0.000000,38.528039,0.0,0.029844,0.000000,0.000000
CS_time0_2014,2.293119,0.529937,0.495085,1.361930,1.078642,0.563003,1.026810,0.618409,0.858803,1.293119,...,138.256017,6.642696,60.868062,1.184399,0.262894,0.000000,0.0,0.880047,0.000000,0.000000
CS_time10_2012,1.701519,0.908847,0.124218,6.172475,1.798034,0.141197,1.705094,0.134942,56.835567,0.456658,...,210.242434,21.346971,14.397728,0.000000,0.000000,4.236117,0.0,0.000000,68.386685,0.087802
CS_time10_2013,1.420912,0.709562,0.271671,2.251117,0.851653,0.204647,1.277033,0.285970,1.787310,0.618409,...,5721.521059,131.340072,67.179783,4.141035,0.000000,4.742720,0.0,6.002407,0.000000,0.000000


In [58]:
for i in range(len(trains_index)):

    Xtrain_mets = data_mets_noreps.iloc[trains_index[i], :]
    
    Xtest_mets = data_mets_noreps.iloc[tests_index[i], :]
    
    vt = VarianceThreshold(0).fit(Xtrain_mets)
    X_train_filtered = vt.transform(Xtrain_mets)
    X_test_filtered = vt.transform(Xtest_mets)
    cols_inds_vt = vt.get_support(indices=True)
    X_train_filtered_df = pd.DataFrame(X_train_filtered, index=Xtrain_mets.index, columns=Xtrain_mets.columns[cols_inds_vt])
    X_test_filtered_df = pd.DataFrame(X_test_filtered, index=Xtest_mets.index, columns=Xtest_mets.columns[cols_inds_vt])

    scaler_model = StandardScaler().fit(X_train_filtered_df)
    X_train_scaled = scaler_model.transform(X_train_filtered_df)
    X_test_scaled = scaler_model.transform(X_test_filtered_df)

    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered_df.columns, index=X_train_filtered_df.index)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered_df.columns, index=X_test_filtered_df.index)

    X_train_scaled_df.to_csv(os.path.join(data_source, 'XTRAIN_METABOLOMICS_NOREPS_SPLIT_'+ str(i) +'.csv'))
    X_test_scaled_df.to_csv(os.path.join(data_source, 'XTEST_METABOLOMICS_NOREPS_SPLIT_'+ str(i) +'.csv'))

## FLUXOMICS

In [62]:
data_fluxes = pd.read_csv(os.path.join(data_source, 'fluxomics_data.csv'), index_col=0)
data_fluxes = data_fluxes.fillna(0)
data_fluxes = data_fluxes.transpose()
data_fluxes.shape

(73, 8632)

In [63]:
for i in range(len(trains_index)):
    
    Xtrain_fluxes = data_fluxes.iloc[trains_index[i], :]
    
    Xtest_fluxes = data_fluxes.iloc[tests_index[i], :]
    
    y_train = ys_train[i]

    # remove some features
    vt = VarianceThreshold(0.1)
    filter_train = vt.fit(Xtrain_fluxes)
    
    train_filtered = filter_train.transform(Xtrain_fluxes)
    test_filtered = filter_train.transform(Xtest_fluxes)
    
    cols_inds = vt.get_support(indices=True)
    
    X_train_filtered = pd.DataFrame(train_filtered, index=Xtrain_fluxes.index, columns=Xtrain_fluxes.columns[cols_inds])
    X_test_filtered = pd.DataFrame(test_filtered, index=Xtest_fluxes.index, columns=Xtest_fluxes.columns[cols_inds])

    kb2 = SelectKBest(f_classif, k=500)

    kb2_fit = kb2.fit(X_train_filtered, y_train)
    
    train_filtered2 = kb2_fit.transform(X_train_filtered)
    test_filtered2 = kb2_fit.transform(X_test_filtered)
    
    cols_inds = kb2_fit.get_support(indices=True)
    
    X_train_filtered2 = pd.DataFrame(train_filtered2, columns=X_train_filtered.columns[cols_inds], index=X_train_filtered.index)
    X_test_filtered2 = pd.DataFrame(test_filtered2, columns=X_test_filtered.columns[cols_inds], index=X_test_filtered.index)
    
    scaler_model = StandardScaler().fit(X_train_filtered2)
    X_train_scaled = scaler_model.transform(X_train_filtered2)
    X_test_scaled = scaler_model.transform(X_test_filtered2)
    
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train_filtered2.columns, index=X_train_filtered2.index)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test_filtered2.columns, index=X_test_filtered2.index)
    
    X_train_scaled_df.to_csv(os.path.join(data_source, 'XTRAIN_FLUXOMICS_REACTIONS_SPLIT_'+ str(i) +'.csv'))
    X_test_scaled_df.to_csv(os.path.join(data_source, 'XTEST_FLUXOMICS_REACTIONS_SPLIT_'+ str(i) +'.csv'))